在深度学习中训练网络时，往往会出现训练数据过于庞大从而无法全部加载到内存中的情况，这里讲述的就是如何分批训练一个庞大的数据集，下面会以Keras 中的训练为例进行讲述。

分批处理的思路就是先将那个较大的数据处理成若干个较小的数据文件（如共1000000 条记录，处理成 1000 个小文件，每个小文件 1000 条记录），然后依次读取各个小的数据文件到内存中进行训练，这里的利用了 python 的 generator 特性来依次读取各个文件的内容。

如下代码所示,就是每次读取 num_files 个文件并合并成 X_train 和 Y_train 并返回，直到整个目录下的文件都被遍历一遍。

In [ ]:
def train_batch_generator(train_data_dir = './processed_data/train/', num_files = 1):
    files = sorted(os.listdir(train_data_dir))
    count = num_files
    embeddings, labels = [], []
    for file in files:
        print('Reading file {0}...........'.format(file))
        gc.collect()
        with open(train_data_dir + file, 'rb') as rf:
            data = pickle.load(rf)
        embeddings.append(data['embedding'])
        labels.append(data['label'])
        count -= 1
        if count == 0: 
            X_train, Y_train = np.concatenate(embeddings), np.concatenate(labels)
            gc.collect()
            count = num_files
            embeddings, labels = [], []
            yield (X_train, Y_train)

这样读取文件对应的训练方法如下（以Keras中的模型训练为例进行说明）

In [ ]:
NUM_EPOCHS = 10BATCH_SIZE = 32

for i in range(NUM_EPOCHS):
    print('################{0} epochs#############'.format(i+1))
    for x_train, y_train in train_batch_generator(num_files = 3):
        print(x_train.shape, y_train.shape)
        gc.collect()
        model.fit(x_train, y_train, batch_size = BATCH_SIZE, epochs = 100, validation_data = (x_test, y_test))